In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
403,Blackwater Valley Exorcism is set on a small t...,negative
426,Well you take O.J. Simpson as a all american s...,positive
325,I was really disappointed by this movie. Great...,negative
952,Hey what do you expect form a very low budget ...,positive
481,This is not an entirely bad movie. The plot (n...,negative


In [4]:
import nltk
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\govin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# data preprocessing

In [5]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)
df.head()

,review,sentiment
403,blackwater valley exorcism set small town ranc...,negative
426,well take o j simpson american soldier turned ...,positive
325,really disappointed movie great actor it poten...,negative
952,hey expect form low budget movie although seen...,positive
481,entirely bad movie plot new house built next d...,negative


In [7]:
df['sentiment'].value_counts()

sentiment
positive    257
negative    243
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
403,blackwater valley exorcism set small town ranc...,0
426,well take o j simpson american soldier turned ...,1
325,really disappointed movie great actor it poten...,0
952,hey expect form low budget movie although seen...,1
481,entirely bad movie plot new house built next d...,0


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [11]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [12]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
import dagshub

mlflow.set_tracking_uri('tracking-uri')
dagshub.init(repo_owner='govin-raaj', repo_name='Movie_sentiment_analysis', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\anaconda\ana\envs\atlas\lib\site-packages\rich\live.py:229: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=fb0eef44-8bdf-4121-a8e8-80638392490c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=34cf1bf5cbeb0d2faf34573bea7dbc42e620038d1aeb3948b156fe7951432189




Accessing as govin-raaj

Initialized MLflow to track repo "govin-raaj/Movie_sentiment_analysis"

Repository govin-raaj/Movie_sentiment_analysis initialized!

2025/09/05 22:35:36 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/9ae01fe40bfe4da38aa147fa9f7248c8', creation_time=1757091942233, experiment_id='0', last_update_time=1757091942233, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-09-05 22:37:33,057 - INFO - Starting MLflow run...
2025-09-05 22:37:33,618 - INFO - Logging preprocessing parameters...
2025-09-05 22:37:34,931 - INFO - Initializing Logistic Regression model...
2025-09-05 22:37:34,932 - INFO - Fitting the model...
2025-09-05 22:37:34,943 - INFO - Model training complete.
2025-09-05 22:37:34,943 - INFO - Logging model parameters...
2025-09-05 22:37:35,377 - INFO - Making predictions...
2025-09-05 22:37:35,378 - INFO - Calculating evaluation metrics...
2025-09-05 22:37:35,383 - INFO - Logging evaluation metrics...
2025-09-05 22:37:37,041 - INFO - Saving and logging the model...
2025/09/05 22:37:47 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-09-05 22:37:47,168 - INFO - Model training and logging completed in 13.55 seconds.
2025-09-05 22:37:47,168 - INFO - Accuracy: 0.712
2025-09-05 22:37:47,169 - INFO - Precision: 0.7352941176470589
2025-09